In [5]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [6]:
df = pd.read_csv("glassdoor_datascience_salaries_cleaned.csv")

In [7]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,job_state,job_city,same_state_as_hq,company_age,python_yn,spark_yn,cloud_yn,deployments_yn,viz_tools_yn,api_dev_yn
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,NM,Albuquerque,False,51,True,False,False,False,True,False
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,MD,Linthicum,False,40,True,False,False,False,False,False
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,FL,Clearwater,True,14,True,True,False,False,False,False
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,WA,Richland,True,59,True,False,False,False,False,False
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,NY,New York,True,26,True,False,False,False,False,False


In [8]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_text', 'job_state', 'job_city', 'same_state_as_hq',
       'company_age', 'python_yn', 'spark_yn', 'cloud_yn', 'deployments_yn',
       'viz_tools_yn', 'api_dev_yn'],
      dtype='object')

In [9]:
print(df['Job Title'].value_counts())

Data Scientist                     131
Data Engineer                       53
Senior Data Scientist               34
Data Analyst                        15
Senior Data Engineer                14
                                  ... 
Data Scientist/ML Engineer           1
Sr. Data Scientist                   1
Data Engineer 4 - Contract           1
Data Analyst - Asset Management      1
Machine Learning Engineer (NLP)      1
Name: Job Title, Length: 264, dtype: int64


In [10]:
# Job title and seniority

def job_title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    elif 'Gen AI' in title.lower():
        return 'AI engineer'
    else:
        return 'na'
    

def seniority (title):
    if 'sr' in title.lower() or 'sr.' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'junior'
    else:
        return 'na'
    


In [11]:
df['job_title_simplified'] = df['Job Title'].apply(job_title_simplifier)


In [13]:
df.job_title_simplified.value_counts()

data scientist    279
data engineer     119
analyst           102
manager            22
mle                22
director           14
Name: job_title_simplified, dtype: int64